### Importing the Necessary Libraries

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow
from tensorflow import convert_to_tensor
from tensorflow import keras
from tensorflow_datasets import as_numpy
from keras import Input
from keras.models import save_model
from keras.applications import DenseNet121
from keras.models import Model
from keras.utils import image_dataset_from_directory, load_img, split_dataset
from keras.utils.image_utils import img_to_array
from keras.optimizers import Adam
from keras.layers import Rescaling, Resizing
from keras.losses import CategoricalCrossentropy
from keras.metrics import Accuracy, CategoricalAccuracy, AUC
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization

### Splitting the Dataset into Training Set and Test Set

In [14]:
trainingDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    seed = 123,
    subset = 'training',
    validation_split = 0.2
)

testDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/',
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (224, 224),
    seed = 123,
    subset = 'validation',
    validation_split = 0.2
)

trainingDataset = as_numpy(trainingDataset)
print(trainingDataset)

Found 2481 files belonging to 2 classes.
Using 1985 files for training.
Found 2481 files belonging to 2 classes.
Using 496 files for validation.


### Building the DenseNet121 Architecture Model

In [ ]:
denseNet = DenseNet121(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (224, 224, 3)), input_shape = (224, 224, 3), classifier_activation = 'softmax')

denseModel = denseNet.output
denseModel = GlobalAveragePooling2D()(denseModel)
denseModel = Flatten()(denseModel)
denseModel = Dense(units = 512, activation = 'relu')(denseModel)
denseModel = Dropout(rate = 0.25)(denseModel)
denseModel = Dense(units = 2, activation = 'softmax')(denseModel)

model = Model(inputs = denseNet.input, outputs = denseModel)

for layer in denseNet.layers :
    layer.trainable = False

model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = CategoricalCrossentropy(), metrics = [Accuracy(), CategoricalAccuracy(), AUC()])

In [ ]:
model_checkpoint = ModelCheckpoint('./SavedModels/Pretrained_DenseNet121/DenseNet121_BestModel.h5', monitor = 'val_categorical_accuracy', verbose = 1, save_best_only = True, mode = 'auto')

finalModel = model.fit(xTrain, yTrain, batch_size = 32, epochs = 5, verbose = 'auto', callbacks = [model_checkpoint])

save_model(model, './SavedModels/Pretrained_DenseNet121/DenseNet121_BestModel.h5', overwrite = True, include_optimizer = True, save_format = None, signatures = None, options = None)

### Evaluating the Final Model

In [ ]:
loss, accuracy, catAccuracy, auc = model.evaluate(x = xTest, y = yTest, verbose = 'auto', batch_size = 32)
print("Loss: {:.2f} Accuracy: {:.2f} Categorical Accuracy: {:.2f} AUC: {:.2f}".format(loss, accuracy, catAccuracy, auc))

### Making the Predictions

In [ ]:
testImage = load_img(path = '../Datasets/COVID-19_Chest_CT-Scan_Dataset/non-COVID/Non-Covid (1035).png', target_size = (224, 224), color_mode = 'rgb')
testImage = convert_to_tensor(testImage)
testImage = img_to_array(testImage)
testImage = np.array([testImage])
prediction = model.predict(testImage)
print(*prediction[0])